<a href="https://colab.research.google.com/github/Carr-23/StockPred/blob/master/StockPred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yahoofinancials
!pip install yfinance
!pip install --upgrade tensorflow

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-cp37-none-any.whl size=15192 sha256=364aac9cc868a34cb224ee9e8ceaafd32b61129543ccd63531fade8da3a97b82
  Stored in directory: /root/.cache/pip/wheels/d9/7e/cf/4977a8572d5247242a4b13018d1d36923024ba84236e0d28bc
Successfully built yahoofinancials
     |████████████████████████████████| 5.5MB 6.6MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=a40c872bd2a1bff03e62c65a78575ae86cee257e5f8f02d375516e6ed1c65879
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.7/dist-packages (2.4.1)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, LSTM, Dropout

In [ ]:
# Get Data
amd_stock = yf.Ticker('AMD')
amd_historyAll = amd_stock.history(period="5y").reset_index()

In [ ]:
# Delete date since numbers are just fine
amd_historyAll.pop('Date')

0      2016-04-18
1      2016-04-19
2      2016-04-20
3      2016-04-21
4      2016-04-22
          ...    
1254   2021-04-12
1255   2021-04-13
1256   2021-04-14
1257   2021-04-15
1258   2021-04-16
Name: Date, Length: 1259, dtype: datetime64[ns]

In [ ]:
# Round Data to be divisible by 50
rounded = 50 * round(len(amd_historyAll)/50)
limitOfDelete = len(amd_historyAll) - rounded
tempList = list(range(0,limitOfDelete))

This is if we turn to numpy array early

In [ ]:
amd_historyAll = amd_historyAll.drop(amd_historyAll.index[tempList])
amd_historyAll = amd_historyAll.reset_index()
amd_historyAll.pop('index')
amd_historyAll.pop('Dividends')
amd_historyAll.pop('Stock Splits')

amd_historyAllX = amd_historyAll.copy()
amd_historyAllX.pop('Close')

amd_historyAllY = amd_historyAll.copy()
amd_historyAllY.pop('Open')
amd_historyAllY.pop('High')
amd_historyAllY.pop('Low')
amd_historyAllY.pop('Volume')

amd_historyAllX = amd_historyAllX.to_numpy()
amd_historyAllY = amd_historyAllY.to_numpy()

In [ ]:
# Splitting Data by 50 history points

xtrainingAll = []
xtestingAll = []

ytrainingAll = []
ytestingAll = []

for x in range(50, amd_historyAll.shape[0]):
    if x%50 == 0:
      xtestingAll.append(amd_historyAllX[x-50:x])
      ytestingAll.append(amd_historyAllY[x,0])
    else:      
      xtrainingAll.append(amd_historyAllX[x-50:x])
      ytrainingAll.append(amd_historyAllY[x,0])

xHistory = [xtrainingAll,xtestingAll]
yHistory = [ytrainingAll,ytestingAll]

This is if we turn to numpy array later

In [ ]:
# Drop Date (useless information that the model doesn't need to know)
amd_historyAll = amd_historyAll.drop(amd_historyAll.index[tempList])
amd_historyAll = amd_historyAll.reset_index()
amd_historyAll.pop('index')
amd_historyAll.pop('Dividends')
amd_historyAll.pop('Stock Splits')

0       0
1       0
2       0
3       0
4       0
       ..
1245    0
1246    0
1247    0
1248    0
1249    0
Name: Stock Splits, Length: 1250, dtype: int64

In [ ]:
# Creating blank variables
xtrainingAll = pd.DataFrame(columns=['Open', 'High', 'Low','Close', 'Volume'])
xtestingAll = pd.DataFrame(columns=['Open', 'High', 'Low','Close', 'Volume'])

ytrainingAll = pd.DataFrame(columns=['Open', 'High', 'Low','Close', 'Volume'])
ytestingAll = pd.DataFrame(columns=['Open', 'High', 'Low','Close', 'Volume'])

# Splitting Data by 50 history points
for x in range(50, amd_historyAll.shape[0]):
    if x%50 == 0:
      xtrainingAll = xtrainingAll.append(amd_historyAll.loc[x-50:x])
      xtestingAll = xtestingAll.append(amd_historyAll.iloc[x:x+1])
    else:      
      ytrainingAll = ytrainingAll.append(amd_historyAll.loc[x-50:x])
      ytestingAll = ytestingAll.append(amd_historyAll.iloc[x:x+1])

xHistory = [xtrainingAll,xtestingAll]
yHistory = [ytrainingAll,ytestingAll]

# Deleting the useless columns
for x in xHistory:
  x.pop('Close')

for y in yHistory:
  y.pop('Open')
  y.pop('High')
  y.pop('Low')
  y.pop('Volume')

In [ ]:
xtrainingAll1,xtestingAll1 = np.array(xtrainingAll),np.array(xtestingAll)
ytrainingAll1,ytestingAll1 = np.array(ytrainingAll),np.array(ytestingAll)

print(xtrainingAll)
print(ytrainingAll)

# Standarizing Data
std = StandardScaler()
xtrainingAll2 = std.fit_transform(xtrainingAll1)
xtestingAll2 = std.transform(xtestingAll1)

# Normalize, but wont change distribution and weights
scaler = MinMaxScaler(feature_range=(-1,1))
xtrainingAll3 = scaler.fit_transform(xtrainingAll2)
xtestingAll3 = scaler.transform(xtestingAll2)

           Open       High        Low    Volume
0      3.690000   3.690000   3.450000  17717800
1      3.580000   3.750000   3.570000  16972800
2      3.700000   3.710000   3.580000  13784800
3      3.540000   3.670000   3.540000   8688500
4      3.640000   3.800000   3.610000  21529600
...         ...        ...        ...       ...
1196  87.559998  88.330002  85.019997  56631200
1197  86.830002  87.949997  84.660004  42359300
1198  88.489998  89.279999  86.949997  33850000
1199  88.599998  89.480003  87.339996  32312900
1200  88.220001  88.599998  87.059998  30791600

[1224 rows x 4 columns]
          Close
1      3.740000
2      3.600000
3      3.600000
4      3.660000
5      3.680000
...         ...
1245  78.580002
1246  80.190002
1247  78.550003
1248  83.010002
1249  82.150002

[59976 rows x 1 columns]


In [ ]:
print(xtrainingAll1.shape)
print(xtrainingAll)
# print(xtrainingAll)
# print(xtrainingAll1)
# print(xtrainingAll2)
# print(xtrainingAll3)
# print('---------------------------------------')
# print(xtestingAll.shape)
# print(xtestingAll)
# print(xtestingAll1)
# print(xtestingAll2)
# print(xtestingAll3)
# print('---------------------------------------')

AttributeError: ignored

In [ ]:
print(xtrainingAll3.shape)
print(xvalidationAll3.shape)
print(xtestingAll3.shape)

print(ytrainingAll.shape)
print(yvalidationAll.shape)
print(ytestingAll.shape)

(540, 4)
(90, 4)
(150, 4)
(540, 1)
(90, 1)
(150, 1)


In [ ]:
# Print Statements
xtestingAll,xvalidationAll,xtestingAll = np.array(testingAll),np.array(validationAll),np.array(testingAll)
print("--------------------Train------------------------")
print(trainingAll)
print("--------------------Val------------------------")
print(validationAll)
print("--------------------Test------------------------")
print(testingAll)

print("--------------------OG------------------------")
print(amd_historyAll)
print("--------------------OG------------------------")
print(amd_historyAll.iloc[0:4])

--------------------Train------------------------
[[0.00148063 0.00291676 0.00449878 ... 0.14068803 0.         0.        ]
 [0.         0.         0.         ... 0.36741887 0.         0.        ]
 [0.00922551 0.01234013 0.01211211 ... 0.6395115  0.         0.        ]
 ...
 [0.79806374 0.78808609 0.78890295 ... 0.11817898 0.         0.        ]
 [0.76867884 0.77080997 0.7710232  ... 0.13452759 0.         0.        ]
 [0.7493166  0.75005603 0.75441226 ... 0.13940929 0.         0.        ]]
--------------------Val------------------------
[[0.06548975 0.07213372 0.07174991 0.07550834 0.36567459 0.
  0.        ]
 [0.07471527 0.07516267 0.06944284 0.0747087  0.47918127 0.
  0.        ]
 [0.06833712 0.07213372 0.07221132 0.07573681 0.30293066 0.
  0.        ]
 [0.06970387 0.08133273 0.07578727 0.08453279 0.43575475 0.
  0.        ]
 [0.07733485 0.08077183 0.07786364 0.07973498 0.37719173 0.
  0.        ]
 [0.07710706 0.07897688 0.07855578 0.07779301 0.29630766 0.
  0.        ]
 [0.12676538 0